In [2]:
#!apt-get install graphviz
#!pip install graphviz pydot
!pip install imageio

mkl-random 1.0.1 requires cython, which is not installed.
tensorflow 1.11.0 has requirement setuptools<=39.1.0, but you'll have setuptools 40.4.3 which is incompatible.
You are using pip version 10.0.1, however version 18.1 is available.
You should consider upgrading via the 'python -m pip install --upgrade pip' command.


In [1]:
# Imports 
import numpy as np 
import os
import skimage.transform as trans
import matplotlib.pyplot as plt
from keras.models import *
from keras.layers import *
from keras.optimizers import *
from keras.callbacks import ModelCheckpoint, LearningRateScheduler
from keras.preprocessing.image import ImageDataGenerator
from keras import backend
from keras.utils import plot_model

from imageio import imread, imwrite
from skimage import transform

Using TensorFlow backend.


In [2]:
USE_DRIVE = False
if(USE_DRIVE):
    from google.colab import drive
    folder = '/content/gdrive/My Drive/Projects/datasets/steven2358-larynx_data/'
    drive.mount('/content/gdrive')
else:
    folder = 'your local dataset folder here'

In [3]:
def plot_metrics(history):
  # summarize history for accuracy
  plt.plot(history.history['acc'])
  #plt.plot(history.history['val_acc'])
  plt.title('model accuracy')
  plt.ylabel('accuracy')
  plt.xlabel('epoch')
  #plt.legend(['train', 'test'], loc='upper left')
  plt.show()
  # summarize history for loss
  plt.plot(history.history['loss'])
  #plt.plot(history.history['val_loss'])
  plt.title('model loss')
  plt.ylabel('loss')
  plt.xlabel('epoch')
  #plt.legend(['train', 'test'], loc='upper left')
  plt.show()

In [5]:
images = np.zeros((154,160,240,3), dtype=np.uint8)
for i,filename in enumerate(sorted(os.listdir(folder + 'images_cropped/images/'))):
  images[i,:,:,:] = imread(folder + 'images_cropped/images/' + filename, pilmode='RGB')
print("loaded images")
#images = (images - images.mean()) / images.std()
print(images.shape)

labels = np.zeros((154,160,240,3), dtype=np.uint8)
for i,filename in enumerate(sorted(os.listdir(folder + 'labels_cropped/labels/'))):
  labels[i,:,:,:] = imread(folder + 'labels_cropped/labels/' + filename, pilmode='RGB')
print("loaded labels")
#labels = labels / 255
print(labels.shape)

FileNotFoundError: [WinError 3] The system cannot find the path specified: 'your local dataset folder hereimages_cropped/images/'

In [4]:
#Ver que pinta tienen estos arrays:
print(images[3][100][120:140])
print(labels[3][100][120:140])

NameError: name 'images' is not defined

In [0]:
# Alternativa con un generador
data_gen_args = dict(
    rotation_range = 5,
    width_shift_range = 5,
    height_shift_range = 5,
    horizontal_flip = True,
    zoom_range = 0.05,
    data_format = 'channels_last',
    #validation_split = 0.2,
)

image_datagen = ImageDataGenerator(**data_gen_args)
label_datagen = ImageDataGenerator(**data_gen_args)


In [0]:
image_gen = image_datagen.flow(images, seed = 1, batch_size=8, shuffle=False)
label_gen = label_datagen.flow(labels, seed = 1, batch_size=8, shuffle=False)
train_gen = zip(image_gen, label_gen)

In [0]:
# Check that original images and labels match up
position = 55
plt.imshow(images[position])
plt.show()
plt.imshow(labels[position])
plt.show()

In [0]:
# Check that augmented images and labels match up
image_test, label_test = next(train_gen)
image_test, label_test = image_test / 255, label_test / 255
print(image_test[0].dtype)
print(image_test[0][100][100])
print(label_test[0].dtype)
print(label_test[0][100][100])
plt.imshow(image_test[0])
plt.show()
plt.imshow(label_test[0])
plt.show()

In [0]:
checkpoint = ModelCheckpoint(folder + 'unet_checkpoint.hdf5', 
                             monitor='accuracy',
                             verbose=1, 
                             save_best_only=True)

In [0]:
model = unet(input_size=(160,240,3))
#plot_model(model, to_file="model.png")

In [0]:
# Fit numpy arrays
# model.fit(images, labels, epochs=5, batch_size=1, callbacks=[checkpoint], validation_split=0.2, shuffle=True)

In [0]:
model.compile(optimizer = Adam(lr = 1e-4), loss = 'categorical_crossentropy', metrics = ['accuracy'])

In [0]:
# Fit the generator
history = model.fit_generator(train_gen, 
                              steps_per_epoch=154//8,
                              epochs = 10,
                              callbacks = [checkpoint],
                              )
plot_metrics(history)

In [0]:
model.save(folder + 'unet.hdf5')

In [0]:
image, label = next(train_gen)
predicted = model.predict(image[0:1,:,:,:])
print(image[0].shape)
imwrite(folder + 'predicted.png', predicted[0])
imwrite(folder + 'image.png', image[0])
imwrite(folder + 'label.png', label[0])
